In [ ]:
"""
!git clone -b EMT https://github.com/pop756/Quantum_KAN.git
%cd Quantum_KAN
!pip install -r requirements.txt"""

In [8]:
import pennylane as qml
from pennylane import numpy as np
import random

def ising_model_circuit(num_qubits=4,num_layer = 2,single='X',double='ZZ',J=0.25,h=0.125):
    """
    Creates a parameterized quantum circuit for simulating the Ising model
    with specified number of qubits and layers. The circuit includes random 
    interaction strengths and external field strengths.
    
    Args:
    num_qubits (int): Number of qubits in the circuit.
    num_layer (int): Number of layers of Ising interactions and external field terms.
    
    Returns:
    cost (function): The cost function which returns the expected value of the PauliZ observable on the first qubit.
    """
        
    # Define the device
    dev = qml.device("default.mixed", wires=num_qubits)
    
    # Define random interaction strengths and external field strengths
    J = np.array(list([J])*num_qubits)
    h = np.array(list([h])*num_qubits)

    def circuit(params):
        # Apply RX rotations with parameters
        for i in range(num_qubits):
            qml.RX(params[i], wires=i)
        for j in range(num_layer):
            # Apply Ising ZZ interactions
            for i in range(num_qubits - 1):
                getattr(qml,f"Ising{double}")(2 * J[i], wires=[i, i+1])

            
            # Apply external field X terms
            for i in range(num_qubits):
                getattr(qml,f"R{single}")(2 * h[i], wires=i)

    @qml.qnode(dev,interface='torch')
    def cost(params):
        circuit(params)
        return qml.expval(qml.PauliX(0)),qml.expval(qml.PauliX(1)),qml.expval(qml.PauliX(2)),qml.expval(qml.PauliX(3))
    
    return cost



def ising_model_circuit_val(num_qubits=4,num_layer = 2,single='X',double='ZZ',J=0.25,h=0.125):
    """
    Creates a parameterized quantum circuit for simulating the Ising model
    with specified number of qubits and layers. The circuit includes random 
    interaction strengths and external field strengths.
    
    Args:
    num_qubits (int): Number of qubits in the circuit.
    num_layer (int): Number of layers of Ising interactions and external field terms.
    
    Returns:
    cost (function): The cost function which returns the expected value of the PauliZ observable on the first qubit.
    """
        
    # Define the device
    dev = qml.device("default.mixed", wires=num_qubits)
    
    # Define random interaction strengths and external field strengths
    J = np.array(list([J])*num_qubits)
    h = np.array(list([h])*num_qubits)

    def circuit(params):
        # Apply RX rotations with parameters
        for i in range(num_qubits):
            qml.RX(params[i], wires=i)
        for j in range(num_layer):
            # Apply Ising ZZ interactions
            for i in range(num_qubits - 1):
                getattr(qml,f"Ising{double}")(2 * J[i], wires=[i, i+1])

            
            # Apply external field X terms
            for i in range(num_qubits):
                getattr(qml,f"R{single}")(2 * h[i], wires=i)

    @qml.qnode(dev,interface='torch')
    def cost(params):
        circuit(params)
        return qml.expval(qml.PauliX(0)),qml.expval(qml.PauliX(1)),qml.expval(qml.PauliX(2)),qml.expval(qml.PauliX(3))
    
    return cost

# Example usage
num_parameters = 4  # One parameter per qubit for RX rotations
params = list(np.random.uniform(0, 2 * np.pi, num_parameters))
random_circuit = ising_model_circuit(num_parameters ,2,single='Y',double='ZZ',J=0.25,h=0.125)
train_circuit = ising_model_circuit(num_parameters ,2,single='Y',double='ZZ',J=0.,h=0.125)

print(random_circuit(params))
print(qml.draw(random_circuit)(params))


(tensor(-0.21582379, requires_grad=True), tensor(0.22380323, requires_grad=True), tensor(-0.43240336, requires_grad=True), tensor(0.10357427, requires_grad=True))
0: ──RX(1.79)─╭IsingZZ(0.50)──RY(0.25)─────────────────────╭IsingZZ(0.50)──RY(0.25)─────
1: ──RX(4.59)─╰IsingZZ(0.50)─╭IsingZZ(0.50)──RY(0.25)──────╰IsingZZ(0.50)─╭IsingZZ(0.50)
2: ──RX(2.10)────────────────╰IsingZZ(0.50)─╭IsingZZ(0.50)──RY(0.25)──────╰IsingZZ(0.50)
3: ──RX(4.19)───────────────────────────────╰IsingZZ(0.50)──RY(0.25)────────────────────

───────────────────────────┤  <X>
───RY(0.25)────────────────┤  <X>
──╭IsingZZ(0.50)──RY(0.25)─┤  <X>
──╰IsingZZ(0.50)──RY(0.25)─┤  <X>


In [28]:
from functions.Error_mitigation import extra_polation,extra_polation_time
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm import tqdm
from torch.utils.data import random_split, DataLoader
import pennylane as qml
from torch import nn
import copy


# 데이터셋 생성 클래스
class QuantumData_time(Dataset):
    def __init__(self,circuit,meas,input_list = [1, 3, 5],size=50):
        """
        QuantumData_time 클래스는 특정 양자 회로와 측정에 기반하여 데이터를 생성합니다.
        주어진 크기(size)만큼의 데이터를 생성하며, 각 데이터는 일정한 노이즈 요인 하에서
        양자 회로의 결과를 포함합니다.
        
        Args:
        size (int): 생성할 데이터의 수
        """
        np.random.seed(42)
        para_num = len(circuit.device.wires)
        self.data = []
        self.labels = []
        for _ in tqdm(range(size)):
            # 0에서 2π 사이의 랜덤 theta 값 6개 생성
            if _ ==  0:
                theta = [0]*para_num
            else:
                theta = [np.random.uniform(0, 2 * np.pi) for i in range(para_num)]
            # random_circuit 및 meas1 함수와 theta 값을 사용하여 노이즈 팩터와 결과 생성
            noise_factor, result = extra_polation_time(circuit, meas, theta, noise_factor=input_list, p1=0.1, p2=0.1)
            self.data.append(np.array(noise_factor))
            self.labels.append(result)
    
    def __len__(self):
        """
        데이터셋의 크기 반환
        """
        return len(self.data)
    
    def __getitem__(self, idx):
        """
        주어진 인덱스에 해당하는 데이터 및 레이블 반환
        
        Args:
        idx (int): 인덱스
        
        Returns:
        tuple: 데이터 (torch.tensor)와 레이블 (torch.tensor)
        """
        return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.float32)

# 데이터셋 생성 클래스
class QuantumData_gate(Dataset):
    def __init__(self,circuit,meas,input_list = [1, 1.2, 1.4],size=50):
        """
        QuantumData_gate 클래스는 특정 양자 회로와 측정에 기반하여 데이터를 생성합니다.
        주어진 크기(size)만큼의 데이터를 생성하며, 각 데이터는 일정한 노이즈 요인 하에서
        양자 회로의 결과를 포함합니다.
        
        Args:
        size (int): 생성할 데이터의 수
        """
        np.random.seed(42)
        para_num = len(circuit.device.wires)
        self.data = []
        self.labels = []
        for _ in tqdm(range(size)):
            # 0에서 2π 사이의 랜덤 theta 값 6개 생성
            theta = [np.random.uniform(0, 2 * np.pi) for i in range(para_num)]
            # random_circuit 및 meas1 함수와 theta 값을 사용하여 노이즈 팩터와 결과 생성
            noise_factor, result = extra_polation(circuit, meas, theta, noise_factor=input_list, p1=0.05, p2=0.05)
            self.data.append(np.array(noise_factor))
            self.labels.append(result)
    
    def __len__(self):
        """
        데이터셋의 크기 반환
        """
        return len(self.data)
    
    def __getitem__(self, idx):
        """
        주어진 인덱스에 해당하는 데이터 및 레이블 반환
        
        Args:
        idx (int): 인덱스
        
        Returns:
        tuple: 데이터 (torch.tensor)와 레이블 (torch.tensor)
        """
        return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.float32)

# 데이터셋 생성 클래스
class QuantumData_time(Dataset):
    def __init__(self,circuit,meas,input_list = [1, 3, 5],size=50):
        """
        QuantumData_time 클래스는 특정 양자 회로와 측정에 기반하여 데이터를 생성합니다.
        주어진 크기(size)만큼의 데이터를 생성하며, 각 데이터는 일정한 노이즈 요인 하에서
        양자 회로의 결과를 포함합니다.
        
        Args:
        size (int): 생성할 데이터의 수
        """
        np.random.seed(42)
        para_num = len(circuit.device.wires)
        self.data = []
        self.labels = []
        for _ in tqdm(range(size)):
            # 0에서 2π 사이의 랜덤 theta 값 6개 생성
            if _ ==  0:
                theta = [0]*para_num
            else:
                theta = [np.random.uniform(0, 2 * np.pi) for i in range(para_num)]
            # random_circuit 및 meas1 함수와 theta 값을 사용하여 노이즈 팩터와 결과 생성
            noise_factor, result = extra_polation_time(circuit, meas, theta, noise_factor=input_list, p1=0.1, p2=0.1)
            self.data.append(np.array(noise_factor))
            self.labels.append(result)
    
    def __len__(self):
        """
        데이터셋의 크기 반환
        """
        return len(self.data)
    
    def __getitem__(self, idx):
        """
        주어진 인덱스에 해당하는 데이터 및 레이블 반환
        
        Args:
        idx (int): 인덱스
        
        Returns:
        tuple: 데이터 (torch.tensor)와 레이블 (torch.tensor)
        """
        return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.float32)



input_list1 = [0,0.25,0.5,0.75,9]
input_list2 = [1,1.5,2,2.5,3]


from kan import KAN
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.KAN = KAN([len(input_list2),1,1],grid=3)
    def forward(self, x):
        output = self.KAN(x)
        #output = nn.Sigmoid()(output)
        output = torch.squeeze(output)
        return output

model = Model()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
criterion = nn.MSELoss()

meas_val = qml.Hamiltonian([1,1,1], [qml.PauliY(1),qml.PauliY(0),qml.PauliZ(1)],grouping_type='qwc')


#val_dataset = QuantumData_time(random_circuit,meas_val,input_list2,size=50)
Pauli_set = ['X','Y','Z']


models = {}
for i in range(num_parameters):
    for pauli in Pauli_set:
        globals()['model_i_pauli'] = Model()
        optimizer = torch.optim.Adam(globals()['model_i_pauli'].parameters(), lr=0.05)
        criterion = nn.MSELoss()
        meas_train  = qml.Hamiltonian([1],[getattr(qml,f'Pauli{pauli}')(i)])
        # 데이터셋 및 데이터 로더 생성
        train_dataset = QuantumData_time(train_circuit,meas_train,input_list2,size=50)
        
        #val_dataset_temp = QuantumData_time(random_circuit,meas1,input_list1,size=50)
        # 데이터 로더
        train_loader = DataLoader(train_dataset, batch_size=500, shuffle=True)



        from functions.training import Early_stop_train_KAN
        train_seq_KAN = Early_stop_train_KAN(globals()['model_i_pauli'],optimizer,criterion)
        train_seq_KAN.train_model(train_loader,epochs=80,res=1000)
        models[f'{i}_{pauli}'] = globals()['model_i_pauli']

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [00:26<00:00,  1.91it/s]
C:\Users\pop75\AppData\Local\Temp\ipykernel_19628\3199084214.py:138: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.float32)


Epoch 80 Loss 0.000092 acc : 0.000000 reg : 0.000000 stop count : 79

100%|██████████| 50/50 [00:18<00:00,  2.77it/s]


Epoch 80 Loss 0.000103 acc : 0.020000 reg : 0.000000 stop count : 79

...

Epoch 80 Loss 0.000094 acc : 0.000000 reg : 0.000000 stop count : 79

100%|██████████| 50/50 [00:15<00:00,  3.16it/s]


Epoch 80 Loss 0.000111 acc : 0.020000 reg : 0.000000 stop count : 79

100%|██████████| 50/50 [00:14<00:00,  3.36it/s]


Epoch 80 Loss 0.000110 acc : 0.000000 reg : 0.000000 stop count : 79

In [101]:
train_seq_KAN.loss_list

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [32]:
meas_list = [qml.Hamiltonian([1], [qml.PauliY(1)],grouping_type='qwc'),qml.Hamiltonian([1], [qml.PauliY(2)],grouping_type='qwc'),qml.Hamiltonian([1], [qml.PauliY(3)],grouping_type='qwc')]

In [35]:

# 데이터셋 생성 클래스
class QuantumData_time_val(Dataset):
    def __init__(self,circuit,meas_list,input_list = [1, 3, 5],size=50):
        """
        QuantumData_time 클래스는 특정 양자 회로와 측정에 기반하여 데이터를 생성합니다.
        주어진 크기(size)만큼의 데이터를 생성하며, 각 데이터는 일정한 노이즈 요인 하에서
        양자 회로의 결과를 포함합니다.
        
        Args:
        size (int): 생성할 데이터의 수
        """
        np.random.seed(42)
        para_num = len(circuit.device.wires)
        self.data = []
        self.labels = []
        for _ in tqdm(range(size)):
            # 0에서 2π 사이의 랜덤 theta 값 6개 생성
            if _ ==  0:
                theta = [0]*para_num
            else:
                theta = [np.random.uniform(0, 2 * np.pi) for i in range(para_num)]
            # random_circuit 및 meas1 함수와 theta 값을 사용하여 노이즈 팩터와 결과 생성
            for meas in meas_list:
                noise_factor, result = extra_polation_time(circuit, meas, theta, noise_factor=input_list, p1=0.1, p2=0.1)
                self.data.append(np.array(noise_factor))
                self.labels.append(result)
        
    
    def __len__(self):
        """
        데이터셋의 크기 반환
        """
        return len(self.data)
    
    def __getitem__(self, idx):
        """
        주어진 인덱스에 해당하는 데이터 및 레이블 반환
        
        Args:
        idx (int): 인덱스
        
        Returns:
        tuple: 데이터 (torch.tensor)와 레이블 (torch.tensor)
        """
        return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.float32)

In [36]:
val_dataset = QuantumData_time_val(random_circuit,meas_list,input_list2,size=50)

100%|██████████| 50/50 [01:48<00:00,  2.16s/it]


In [83]:
labels = val_dataset.labels

In [60]:
list_0 = [3*i for i in range(50)]
list_1 = [3*i+1 for i in range(50) ]
list_2 = [3*i+2 for i in range(50) ]

In [111]:
data_1 = torch.tensor(np.array((val_dataset.data))[list_0],dtype=torch.float32)
data_2 = torch.tensor(np.array((val_dataset.data))[list_1],dtype=torch.float32)
data_3 = torch.tensor(np.array((val_dataset.data))[list_2],dtype=torch.float32)

label_1 = torch.tensor(labels)[list_0]
label_2 = torch.tensor(labels)[list_1]
label_3 = torch.tensor(labels)[list_2]

In [82]:
from torch import nn
output_1 = models['1_Y'](data_1)
output_2 = models['2_Y'](data_2)
output_3 = models['3_Y'](data_3)

In [110]:
nn.MSELoss()(output_1,label_1)

tensor(0.1400, dtype=torch.float64, grad_fn=<MseLossBackward0>)

In [112]:
nn.MSELoss()(output_2,label_2)

tensor(0.0966, dtype=torch.float64, grad_fn=<MseLossBackward0>)

In [113]:
nn.MSELoss()(output_3,label_3)

tensor(0.1457, dtype=torch.float64, grad_fn=<MseLossBackward0>)